# Nest Neural Network

In [8]:
"""
Response Modeling with Neural Network (PyTorch)
Author: Davide Rossetti, PhD
Description:
This script trains a nested neural network model to approximate a dynamic response
from simulated data (e.g., memristive system). The network uses two internal
modules (model1 and model4) to capture nonlinear dependencies.
"""

# -------------------------------------------------------------------------
# 1. Imports
# -------------------------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------------------------------------
# 2. Load Dataset and Labels
# -------------------------------------------------------------------------
# Files must contain numeric values separated by spaces.
# Example: Dataset_R250.txt and Label_R250.txt were generated with r = 2.5.

print("Loading dataset...")

with open("Dataset_R250.txt", "r") as file:
    dataset = [list(map(float, line.strip().split())) for line in file]

with open("Label_R250.txt", "r") as file:
    labels = [list(map(float, line.strip().split())) for line in file]

X = np.array(dataset)
y = np.array(labels)

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# -------------------------------------------------------------------------
# 3. Train/Test Split
# -------------------------------------------------------------------------
dim_test = 100000  # number of test samples

X_train = X[dim_test:len(X)]
X_test = X[len(X) - dim_test:]
y_train = y[dim_test:len(y)]
y_test = y[len(y) - dim_test:]

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# -------------------------------------------------------------------------
# 4. Define Neural Network
# -------------------------------------------------------------------------
# Network with two nested models:
# - model1: nonlinear local processing block
# - model4: final linear reduction to 3 output channels
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Local nonlinear transformation applied to each input variable
        self.model1 = nn.Sequential(
            nn.Linear(1, 2),
            nn.Identity(),
            nn.Linear(2, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
            nn.Identity(),
        )

        # Final stage that combines original and processed features
        self.model4 = nn.Sequential(
            nn.Linear(6, 3),
            nn.Identity(),
        )

    def forward(self, x):
        # Apply nonlinear subnetwork to each column
        out1 = self.model1(x[:, 0].unsqueeze(1))
        out2 = self.model1(x[:, 1].unsqueeze(1))
        out3 = self.model1(x[:, 2].unsqueeze(1))

        # Concatenate original inputs with the three outputs
        combined = torch.cat((x, out1, out2, out3), dim=1)

        # Final mapping
        out_final = self.model4(combined)
        return out_final


# -------------------------------------------------------------------------
# 5. Model Setup
# -------------------------------------------------------------------------
model = NeuralNetwork()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("\nModel initialized:")
print(model)

# -------------------------------------------------------------------------
# 6. Training Loop
# -------------------------------------------------------------------------
epochs = 10
batch_size = 256
l1_lambda = 0.0001  # L1 regularization strength

loss_history = []

print("\nStarting training...")

for epoch in range(epochs):
    running_loss = 0.0

    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # L1 regularization term
        l1_reg = torch.tensor(0.)
        for param in model.parameters():
            l1_reg += torch.abs(param).sum()
        loss += l1_lambda * l1_reg

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / (len(X_train) / batch_size)
    loss_history.append(avg_loss)
    print(f"Epoch {epoch + 1}/{epochs} - Average Loss: {avg_loss:.6f}")

print("\nTraining completed successfully.")

# -------------------------------------------------------------------------
# 7. Evaluation on Test Data
# -------------------------------------------------------------------------
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = loss_fn(y_pred_test, y_test)
    print(f"\nTest MSE Loss: {test_loss.item():.6f}")

# Compute Mean Absolute Percentage Error (MAPE)
y_true_np = y_test.numpy()
y_pred_np = y_pred_test.numpy()
mape = np.mean(np.abs((y_true_np - y_pred_np) / (y_true_np + 1e-8))) * 100
print(f"Test MAPE: {mape:.3f}%")


print("\nModel and loss history saved successfully.")


Loading dataset...
Dataset shape: (1000010, 3)
Labels shape: (1000010, 3)
Training set: torch.Size([900010, 3]), Test set: torch.Size([100000, 3])

Model initialized:
NeuralNetwork(
  (model1): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Identity()
    (2): Linear(in_features=2, out_features=2, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2, out_features=1, bias=True)
    (5): Identity()
  )
  (model4): Sequential(
    (0): Linear(in_features=6, out_features=3, bias=True)
    (1): Identity()
  )
)

Starting training...
Epoch 1/10 - Average Loss: 0.014738
Epoch 2/10 - Average Loss: 0.005166
Epoch 3/10 - Average Loss: 0.002303
Epoch 4/10 - Average Loss: 0.001985
Epoch 5/10 - Average Loss: 0.002025
Epoch 6/10 - Average Loss: 0.002002
Epoch 7/10 - Average Loss: 0.002015
Epoch 8/10 - Average Loss: 0.002008
Epoch 9/10 - Average Loss: 0.002011
Epoch 10/10 - Average Loss: 0.002009

Training completed successfully.

Test MSE Loss: 0.002835
Test MAPE: